In [84]:
import pandas as pd
import numpy as np
import keras
import keras.regularizers as regularizers
import keras.optimizers as optimizers
import keras.models as models
import keras.layers as layers

In [61]:
raw=pd.read_csv("NIFTY50.csv")

In [62]:
raw=(raw).dropna()
raw=raw.reset_index(drop=True)

In [278]:
"""To predict prices we are not going to just create a final output neuron with requiered value(High or Low)
but to obtain more accurate results we take the ratio of high/open or open/low subtract 1 and multiply by 1000000.
And this will be the output of our neural model. With this ratio and day's open we can calculate day's high and low value."""

"To predict prices we are not going to just create a final output neuron with requiered value(High or Low)\nbut to obtain more accurate results we take the ratio of high/open or open/low subtract 1 and multiply by 1000000.\nAnd this will be the output of our neural model. With this ratio and day's open we can calculate day's high and low value."

In [63]:
Data=pd.DataFrame()
Data['High']=(raw['High']/raw['Open']-1)*1000000

In [64]:
Data['open1']=0.0
Data['open2']=0.0
Data['open3']=0.0
Data['open4']=0.0
Data['open5']=0.0
Data['open6']=0.0
Data['open7']=0.0

In [65]:
for i in range(len(raw['Open'])):
    if i>7:
        for k in range(1,8,1):
            Data[str('open'+str(k))][i]=raw['Open'][(i-k)]

In [75]:
Data=Data[Data['open1']!=0.0]

In [77]:
Data=Data.reset_index(drop=True)

In [78]:
Data

,High,open1,open2,open3,open4,open5,open6,open7
0,11878.355169,4942.700195,4937.600098,4939.100098,4837.149902,4752.950195,4734.850098,4550.250000
1,13501.902818,4996.450195,4942.700195,4937.600098,4939.100098,4837.149902,4752.950195,4734.850098
2,37946.359834,5021.500000,4996.450195,4942.700195,4937.600098,4939.100098,4837.149902,4752.950195
3,4115.816757,5069.000000,5021.500000,4996.450195,4942.700195,4937.600098,4939.100098,4837.149902
4,7757.054570,5211.649902,5069.000000,5021.500000,4996.450195,4942.700195,4937.600098,4939.100098
...,...,...,...,...,...,...,...,...
3160,5158.846778,11359.599609,11354.400391,11566.200195,11478.549805,11464.299805,11777.549805,11602.950195
3161,7090.972561,11378.549805,11359.599609,11354.400391,11566.200195,11478.549805,11464.299805,11777.549805
3162,8866.262594,11218.599609,11378.549805,11359.599609,11354.400391,11566.200195,11478.549805,11464.299805
3163,3992.050506,11363.299805,11218.599609,11378.549805,11359.599609,11354.400391,11566.200195,11478.549805


In [220]:
nn=models.Sequential()

In [221]:
nn.add(layers.Dense(800,activation='relu',kernel_regularizer=regularizers.l1(1000),input_shape=(7,)))
nn.add(layers.Dense(3000,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
nn.add(layers.Dense(5000,activation='relu',kernel_regularizer=regularizers.l1(1000),input_shape=(7,)))
#nn.add(layers.Dense(1000,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
nn.add(layers.Dense(3000,activation='relu',kernel_regularizer=regularizers.l1(1000),input_shape=(7,)))
#nn.add(layers.Dense(50,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
nn.add(layers.Dense(1,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))

In [222]:
def los(a,b):
    return abs(a-b)*2

In [225]:
nn.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['accuracy'])

In [226]:
Input=Data.iloc[:2500,1:]
Output=Data.iloc[:2500,0:1]
Input_t=Data.iloc[2500:,1:]
Output_t=Data.iloc[2500:,0:1]

In [239]:
nn.fit(Input,Output,batch_size=100,epochs=25)

Epoch 1/25
2500/2500 [==============================] - 4s 2ms/step - loss: 130836260.9600 - accuracy: 0.0000e+00
Epoch 2/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131463188.4800 - accuracy: 0.0000e+00
Epoch 3/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131149592.4800 - accuracy: 0.0000e+00
Epoch 4/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131490222.7200 - accuracy: 0.0000e+00
Epoch 5/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131093382.0800 - accuracy: 0.0000e+00
Epoch 6/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131613092.8000 - accuracy: 0.0000e+00
Epoch 7/25
2500/2500 [==============================] - 4s 1ms/step - loss: 130875335.2000 - accuracy: 0.0000e+00
Epoch 8/25
2500/2500 [==============================] - 4s 1ms/step - loss: 131087597.1200 - accuracy: 0.0000e+00
Epoch 9/25
2500/2500 [==============================] - 4s 1ms/step - loss: 130964583.04

In [280]:
"""Testing on test set."""
for i in range(500):
    print(raw['High'][(i+8+2500)]-((nn.predict(Input_t.iloc[8:(8+1),:])[0])/1000000+1)*raw['Open'][(i+2500+8)])

[-78.57496669]
[-63.2642439]
[-53.20775368]
[-81.76751937]
[-78.5170891]
[-55.74097335]
[-68.10630925]
[-61.00104363]
[-86.95695052]
[-97.62348557]
[-73.38154666]
[-93.5124987]
[-37.55381288]
[-101.26260727]
[-1.00210774]
[-89.07439205]
[-25.10518641]
[-11.36757406]
[-8.85059491]
[-64.28329659]
[-105.28237392]
[-12.92798719]
[-105.27196259]
[-64.85345168]
[-31.99817568]
[-87.03886437]
[-2.17040204]
[201.31281668]
[-93.84831286]
[19.49333315]
[-35.13843352]
[-62.50382211]
[-95.64477925]
[-19.79133757]
[-83.8435466]
[-99.07639322]
[-60.24705868]
[-98.82916179]
[-55.04870858]
[-7.75892006]
[-33.48246372]
[-84.27383114]
[-52.9762788]
[-53.89088085]
[-98.55020286]
[-78.02697416]
[-86.6975728]
[-42.83891532]
[-72.06048321]
[34.30240424]
[-9.48282569]
[-71.31631359]
[-84.18397567]
[-97.01220233]
[-87.72978482]
[8.72484586]
[-51.26345513]
[-56.12634314]
[-35.69016352]
[59.56595569]
[-76.77007867]
[-81.49916945]
[-27.87634259]
[-28.159554]
[-92.24218393]
[6.29532217]
[-70.24887163]
[-34.0529693

In [272]:
print((nn.predict(pd.DataFrame([[11487.20,11540.15,11447.80,11363.3,11218.6,11378.55,11359.60]]))/1000000+1))

[[1.01023939]]


In [249]:
nn.save('HighValuePredictor')

In [260]:
Data2=pd.DataFrame()
Data2['Low']=(raw['Open']/raw['Low']-1)*1000000
for k in range(7):
    Data2['open'+str(k+1)]=0

In [261]:
for i in range(len(raw['Close'])):
    if i>7:
        for k in range(1,8,1):
            Data2[str('open'+str(k))][i]=raw['Open'][(i-k)]

C:\Users\sampu\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [262]:
Data2=Data2[Data2['open1']!=0.0]

In [263]:
Data2=Data2.reset_index(drop=True)

In [264]:
Data2

,Low,open1,open2,open3,open4,open5,open6,open7
0,0.000000,4942,4937,4939,4837,4752,4734,4550
1,4028.892520,4996,4942,4937,4939,4837,4752,4734
2,6922.737439,5021,4996,4942,4937,4939,4837,4752
3,16699.037320,5069,5021,4996,4942,4937,4939,4837
4,8451.912479,5211,5069,5021,4996,4942,4937,4939
...,...,...,...,...,...,...,...,...
3160,7803.020117,11359,11354,11566,11478,11464,11777,11602
3161,2990.502303,11378,11359,11354,11566,11478,11464,11777
3162,3169.254442,11218,11378,11359,11354,11566,11478,11464
3163,2443.052307,11363,11218,11378,11359,11354,11566,11478


In [266]:
mm=models.Sequential()
mm.add(layers.Dense(500,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.add(layers.Dense(2000,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.add(layers.Dense(3000,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.add(layers.Dense(5000,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.add(layers.Dense(500,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.add(layers.Dense(1,activation='relu',kernel_regularizer=regularizers.l2(1000),input_shape=(7,)))
mm.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['accuracy'])

In [268]:
mm.fit(Data2.iloc[:2500,1:],Data2.iloc[:2500,:1],epochs=25,batch_size=20)

Epoch 1/25
2500/2500 [==============================] - 17s 7ms/step - loss: 901928060.0800 - accuracy: 0.0560 0s - loss: 947830616.4237 - ac
Epoch 2/25
2500/2500 [==============================] - 12s 5ms/step - loss: 438670004.4800 - accuracy: 0.0556
Epoch 3/25
2500/2500 [==============================] - 13s 5ms/step - loss: 240490660.4800 - accuracy: 0.0560
Epoch 4/25
2500/2500 [==============================] - 12s 5ms/step - loss: 1089300451.2000 - accuracy: 0.0068
Epoch 5/25
2500/2500 [==============================] - 12s 5ms/step - loss: 200850294.6080 - accuracy: 0.0000e+00
Epoch 6/25
2500/2500 [==============================] - 12s 5ms/step - loss: 202709927.4880 - accuracy: 0.0000e+00
Epoch 7/25
2500/2500 [==============================] - 13s 5ms/step - loss: 202114330.8320 - accuracy: 0.0000e+00
Epoch 8/25
2500/2500 [==============================] - 12s 5ms/step - loss: 194647941.6800 - accuracy: 0.0000e+00
Epoch 9/25
2500/2500 [==============================] - 12s 5ms/

In [279]:
"""Testing on test set"""
for i in range(2500,3000,1):
    print(raw['Low'][i+8]-(1/(mm.predict(Data2.iloc[i:i+1,1:])[0]/1000000+1))*raw['Open'][i+8]-180)

[-59.7294416]
[-36.08421129]
[-1.38316925]
[-70.44842609]
[-50.64668744]
[-25.80012341]
[-10.77250099]
[1.23518996]
[-36.15177542]
[-53.58285708]
[-18.71675444]
[-78.20252666]
[3.61619954]
[-64.03117567]
[-26.58323561]
[-78.38800251]
[-23.01717996]
[11.79875695]
[13.4028783]
[-4.74365832]
[-66.83658505]
[16.73663844]
[-64.74926149]
[-18.62474797]
[-143.85976354]
[-181.65407248]
[-3.15654523]
[-10.92916763]
[-148.41205897]
[-29.92696378]
[-13.40324035]
[-27.20946164]
[-123.03505861]
[-23.29753375]
[-158.10489941]
[-182.68399747]
[-41.88442001]
[-74.68243408]
[-13.08138115]
[-10.40359631]
[-3.7633397]
[-73.83390377]
[-25.73871778]
[-31.08557983]
[-103.07897404]
[-203.52021531]
[-93.62173635]
[-72.41018129]
[-61.55842607]
[-8.88752611]
[-13.61502094]
[-58.83716908]
[-61.09664147]
[-167.95327784]
[-144.94609184]
[-10.01522473]
[-54.58703833]
[-68.39455077]
[-27.38707143]
[-41.47618572]
[-56.46691293]
[-56.50649678]
[-34.19870429]
[-24.79755959]
[-170.60499887]
[-9.35873315]
[-38.01981122]


In [273]:
print((mm.predict(pd.DataFrame([[11487.20,11540.15,11447.80,11363.3,11218.6,11378.55,11359.60]]))/1000000+1))

[[1.019233]]


In [277]:
mm.save("LowValuePredictor")